In [ ]:
import os
os.chdir('..')
print(os.getcwd())

: 

In [34]:
from utils.mano import ManoLayer
from utils.utils import load_ho_meta, get_obj_mesh, apply_transform_to_mesh, add_trimesh_to_fig
import trimesh
import plotly.graph_objects as go
import trimesh
import numpy as np
import torch
from scipy.spatial.transform import Rotation as R

# load object
obj_mesh = trimesh.load_mesh('data/models/006_mustard_bottle/textured_simple.obj')


class AmbientSensor():
    def __init__(self, cell_length=0.018):
        # 3次元格子の生成
        x = np.linspace(0, cell_length*10, 10)  # 0から1までを10ステップで分割
        y = np.linspace(0, cell_length*10, 10)
        z = np.linspace(0, cell_length*10, 10)
        X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
        self.lattice = torch.from_numpy(np.column_stack((X.ravel(), Y.ravel(), Z.ravel()))).to(torch.float32)
        # print(X.shape)
        # print(self.lattice.shape)
        # self.values = np.zeros_like(self.lattice)
        self.values = torch.rand(self.lattice.shape[0])
        self.cell_length = cell_length


    # set transform to mano hand
    def calc_ambient(self, mano_hand, obj_mesh):
        middle_pos = mano_hand.joints[:, 4]#[np.newaxis, :]
        rotvec = mano_hand.global_orient[0]
        rot = torch.from_numpy(R.from_rotvec(rotvec).as_matrix()).to(torch.float32)
        convert = torch.tensor([[1,0,0],[0,0,1],[0,-1,0]], dtype=torch.float32)

        print(self.lattice.shape, rot.shape, middle_pos.shape)
        print(self.lattice.dtype, rot.dtype, middle_pos.dtype)
        # trans = torch.tensor([[-0.04,-0.09,0.0]])
        trans = torch.tensor([[0.04,0.0,0.09]])
        self.lattice = (self.lattice - trans) @ rot.T @ convert.T + middle_pos
        # self.lattice = (self.lattice + torch.tensor([[-0.04,-0.09,0.0]])) + middle_pos
        print(self.lattice.shape)
        sd = torch.from_numpy(obj_mesh.nearest.signed_distance(self.lattice)).to(torch.float32)
        self.values[sd < (-self.cell_length)] = 0
        self.values[sd > (-self.cell_length)] = 1 - sd[sd > (-self.cell_length)] / self.cell_length
        self.values[sd > 0] = 1
        # self.values = torch.sigmoid(-sd)
        return self.values


    def get_spheres(self):
        # 3次元格子を描画
        lattice_spheres = [trimesh.primitives.Sphere(radius=0.005, center=point) for point in lattice]



mano = ManoLayer()
anno = load_ho_meta()

mano_hand = mano(anno)

obj_verts, obj_faces, obj_uv, obj_map = get_obj_mesh(anno)
obj_verts = apply_transform_to_mesh(obj_verts, anno)


# print(mano_hand.shape, mano.faces.shape)
mano_mesh = trimesh.Trimesh(vertices=mano_hand.vertices[0], faces=mano.faces, )
obj_mesh = trimesh.Trimesh(vertices=obj_verts, faces=obj_faces)

amb = AmbientSensor()
amb.calc_ambient(mano_hand, obj_mesh)
# print(amb.values)
print(amb.values.min(), amb.values.max())

fig = go.Figure()
add_trimesh_to_fig(fig, mano_mesh, opacity=0.8)
add_trimesh_to_fig(fig, obj_mesh, opacity=0.8)
# fig.add_trace(
#     go.Volume(
#         x=amb.lattice[:, 0],
#         y=amb.lattice[:, 1],
#         z=amb.lattice[:, 2],
#         value=amb.values,
#         isomin=0.0,
#         isomax=1.0,
#         opacity=0.5, # needs to be small to see through all surfaces
#         surface_count=17, # needs to be a large number for good volume rendering
#         colorscale='Viridis'
        
#     )
# )
fig.add_trace(
    go.Scatter3d(
        x=mano_hand.joints[:, 4, 0],
        y=mano_hand.joints[:, 4, 1],
        z=mano_hand.joints[:, 4, 2],
        mode='markers',
        marker=dict(
            size=5,
            color='red',
            opacity=1
        )
    )
)

fig.add_trace(
    go.Scatter3d(
        x=amb.lattice[:, 0],
        y=amb.lattice[:, 1],
        z=amb.lattice[:, 2],
        mode='markers',
        marker=dict(
            size=1.5,
            color=amb.values,
            colorscale='Viridis',
            opacity=0.5
        )
    )
)

/mnt/SamsungSSD/workspace/yoshioka/rendering/utils/utils.py:18: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



torch.Size([8251, 3]) torch.Size([3, 3]) torch.Size([3])
torch.Size([1000, 3]) torch.Size([3, 3]) torch.Size([1, 3])
torch.float32 torch.float32 torch.float32
torch.Size([1000, 3])
tensor(0.) tensor(1.9760)
